In [532]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['figure.figsize'] = [9, 8]

import cv2
import numpy as np
import pandas as pd
pd.set_option('display.width', 74)
pd.set_option('display.max_columns', 15)
pd.set_option('display.max_rows', 20)

In [533]:
import os

DATADIR = "data/keras"
NUM_CHARACTERS = len(os.listdir(os.path.join(DATADIR, "train", "images")))

In [539]:
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Input, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.optimizers import Adam

# Image processing
IMAGE_ROW_SIZE = 584
IMAGE_COLUMN_SIZE = 480

def init_model(target_num=4, dropout_ratio=0.5, learning_rate=0.0001):
    input_shape = (IMAGE_ROW_SIZE, IMAGE_COLUMN_SIZE, 3)

    # Fine-tune prediction layer
    pretrained_model = VGG16(include_top=False, weights='imagenet',
                             input_shape=input_shape)
    for layer in pretrained_model.layers:
        layer.trainable = False

    output_tensor = pretrained_model.output
    output_tensor = Flatten()(output_tensor)
    output_tensor = Dense(128, activation='relu')(output_tensor)
    # output_tensor = Dense(128, activation='relu')(output_tensor)
    output_tensor = Dense(target_num, activation="softmax", name="predictions")(output_tensor)

    # Define and compile the model
    model = Model(inputs=pretrained_model.input, outputs=output_tensor)
    model.compile(optimizer=Adam(lr=learning_rate),
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])

    return model

model = init_model(target_num=NUM_CHARACTERS)

In [540]:
from keras.preprocessing.image import ImageDataGenerator

def make_generator(folder="train",
                   data_gen_args={"fill_mode": "constant",
                                  "cval": 0,
                                  "width_shift_range": 0.05,
                                  "height_shift_range": 0.05,
                                  "zoom_range": 0.1,
                                  "horizontal_flip": True,
                                  "rescale": 1.0 / 255},
                   data_flow_args={"seed": 1,
                                   "batch_size": 32}):

    image_datagen = ImageDataGenerator(**data_gen_args)

    image_generator = image_datagen.flow_from_directory(
        directory=os.path.join(DATADIR, folder, "images"),
        target_size=(IMAGE_ROW_SIZE, IMAGE_COLUMN_SIZE),
        color_mode='rgb',
        **data_flow_args)

    return image_generator

def steps_per_epoch(folder="train", batch_size=32):
    image_directory = os.path.join(DATADIR, folder, "images")
    char_name = os.listdir(image_directory)[0]
    char_directory = os.path.join(image_directory, char_name)
    data_size = len(os.listdir(char_directory))
    return data_size // batch_size

In [541]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 584, 480, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 584, 480, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 584, 480, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 292, 240, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 292, 240, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 292, 240, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 146, 120, 128)     0         
__________

In [ ]:
model.fit_generator(generator=make_generator("train"),
                    steps_per_epoch=steps_per_epoch("train"),
                    epochs=10,
                    validation_data=make_generator("valid"),
                    validation_steps=steps_per_epoch("valid"))

Found 78406 images belonging to 5 classes.
Found 9801 images belonging to 5 classes.
Epoch 1/10
475/475 [==============================] - 1072s 2s/step - loss: 1.2433 - acc: 0.4914 - val_loss: 0.9657 - val_acc: 0.6453
Epoch 2/10
475/475 [==============================] - 1046s 2s/step - loss: 0.9030 - acc: 0.6612 - val_loss: 0.8073 - val_acc: 0.6865
Epoch 3/10
475/475 [==============================] - 1042s 2s/step - loss: 0.7801 - acc: 0.7111 - val_loss: 0.7523 - val_acc: 0.7245
Epoch 4/10
475/475 [==============================] - 1046s 2s/step - loss: 0.6906 - acc: 0.7522 - val_loss: 0.7245 - val_acc: 0.7354
Epoch 5/10
475/475 [==============================] - 1053s 2s/step - loss: 0.6242 - acc: 0.7771 - val_loss: 0.5781 - val_acc: 0.8016
Epoch 6/10
475/475 [==============================] - 1056s 2s/step - loss: 0.5589 - acc: 0.8010 - val_loss: 0.5246 - val_acc: 0.8042
Epoch 7/10
475/475 [==============================] - 1053s 2s/step - loss: 0.5185 - acc: 0.8150 - val_loss: 0.

In [10]:
import datetime

model.save(os.path.join(DATADIR, datetime.datetime.today().strftime('%Y-%m-%d') + "-short-epochs-1-hidden-layer.h5"))

In [13]:
def make_test_generator(folder="test",
                        data_flow_args={"seed": 1,
                                        "batch_size": 32}):

    return ImageDataGenerator().flow_from_directory(
        directory=os.path.join(DATADIR, folder, "images"),
        target_size=(IMAGE_ROW_SIZE, IMAGE_COLUMN_SIZE),
        color_mode='rgb',
        **data_flow_args)

In [525]:
test_batches = make_test_generator()
class_dict = test_batches.class_indices
index_dict = {i: c for c, i in class_dict.items()}

Found 9800 images belonging to 5 classes.


In [528]:
images, vectors = next(test_batches)
true_labels = [index_dict[i] for i in vectors.argmax(1)]
predictions = model.predict_on_batch(images)
pred_labels = [index_dict[i] for i in predictions.argmax(1)]

In [531]:
pd.DataFrame({'True': true_labels, 'Predicted': pred_labels})

,Predicted,True
0,marth,marth
1,peach,peach
2,falco,falco
3,marth,fox
4,falco,falco
5,peach,peach
6,marth,fox
7,falco,jigglypuff
8,falco,falco
9,falco,falco
